In [92]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix 
from skimage import feature 
from imutils import build_montages 
from imutils import paths 
import numpy as np 
import cv2 
import os 
import pickle

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics

In [8]:
wave_train_dir = "./dataset/wave/training"
wave_test_dir = "./dataset/wave/testing"

spiral_train_dir = "./dataset/spiral/training"
spiral_test_dir = "./dataset/spiral/testing"

In [21]:
def quantify_image(image):
  features = feature.hog(image, orientations=9, pixels_per_cell=(13, 13), cells_per_block=(4, 4), transform_sqrt=True, block_norm="L1")
  return features

def load_split(path):
  imagePaths = list(paths.list_images(path))
  data = [] 
  labels = []

  for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold (image, 0, 255,
    cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU) [1]
    features = quantify_image(image)
    data.append(features)
    labels.append(label)
  return (np.array(data), np.array(labels))

In [22]:
wave_train_x, wave_train_y = load_split(wave_train_dir)
wave_test_x, wave_test_y = load_split(wave_test_dir)

spiral_train_x, spiral_train_y = load_split(wave_train_dir)
spiral_test_x, spiral_test_y = load_split(spiral_test_dir)


In [23]:
wave_train_x.shape, wave_train_y.shape, "\n", wave_test_x.shape, wave_test_y.shape

((72, 20736), (72,), '\n', (30, 20736), (30,))

In [24]:
spiral_train_x.shape, spiral_train_y.shape, "\n", spiral_test_x.shape, spiral_test_y.shape

((72, 20736), (72,), '\n', (30, 20736), (30,))

In [25]:
le_wave = LabelEncoder() 
le_spiral = LabelEncoder() 

wave_train_y = le_wave.fit_transform(wave_train_y)
wave_test_y = le_wave.transform(wave_test_y)

spiral_train_y = le_spiral.fit_transform(spiral_train_y)
spiral_test_y = le_spiral.transform(spiral_test_y)


In [137]:
waveModel = SVC(kernel="poly", degree=5) 
spiralModel = KNeighborsClassifier(21)

#waveModel = KNeighborsClassifier(n)
#spiralModel = KNeighborsClassifier(21)


In [138]:
waveModel.fit(wave_train_x, wave_train_y)
spiralModel.fit(spiral_train_x, spiral_train_y)

KNeighborsClassifier(n_neighbors=21)

In [139]:
wave_pred_y = waveModel.predict(wave_test_x)
spiral_pred_y = spiralModel.predict(spiral_test_x)

In [141]:
print("Validation Evaluation")
print(metrics.classification_report(wave_pred_y, wave_test_y))

Validation Evaluation
              precision    recall  f1-score   support

           0       0.93      0.78      0.85        18
           1       0.73      0.92      0.81        12

    accuracy                           0.83        30
   macro avg       0.83      0.85      0.83        30
weighted avg       0.85      0.83      0.84        30



In [142]:
print("Validation Evaluation")
print(metrics.classification_report(spiral_pred_y, spiral_test_y))

Validation Evaluation
              precision    recall  f1-score   support

           0       0.60      0.69      0.64        13
           1       0.73      0.65      0.69        17

    accuracy                           0.67        30
   macro avg       0.67      0.67      0.67        30
weighted avg       0.68      0.67      0.67        30



In [143]:
pickle.dump(waveModel, open("./models/waveModel.pkl", "wb"))
pickle.dump(spiralModel, open("./models/spiralModel.pkl", "wb"))